In [1]:
# earth engine
import ee
import geemap.eefolium as geemap
#from geemap import geojson_to_ee, ee_to_geojson

# analysis and mapping
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
#import json
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from pprint import pprint

# file management stuff
import datetime
import glob
import urllib
import zipfile
from zipfile import ZipFile 

ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Add EE drawing method to folium.
#folium.Map.add_ee_layer = add_ee_layer
folium.plugins.DualMap.add_ee_layer = add_ee_layer

In [5]:
# Function to sample values by HOLC zones
def sample_holc_layer(zones, dataset, scale):
  resultList = []
  for feat in zones.getInfo()['features']:
    name = feat['properties']['level_0']
    print(name)
    region = ee.Feature(zones.filter(ee.Filter.eq('level_0', name)).first())
    # Reduce the region. The region parameter is the Feature geometry.
    meanDictionary = dataset.reduceRegion(**{'reducer': ee.Reducer.mean(),'geometry': region.geometry(),'scale': scale,'maxPixels': 1e9})
    # The result is a Dictionary.  Print it.
    #resultDict[name]=meanDictionary.getInfo()
    resultList.append(meanDictionary.getInfo())
    pprint(meanDictionary.getInfo())

  datadf = pd.DataFrame(resultList)
  datadf.reset_index(inplace=True)
  datadf['joinid']=datadf['index'].apply(str)

  return datadf

In [6]:
url = r'https://dsl.richmond.edu/panorama/redlining/static/fullshpfile.zip'
dwnfile = url.split('/')[-1]

# Copy a network object to a local file
urllib.request.urlretrieve(url, dwnfile)

# specifying the zip file name 
file_name = dwnfile
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zf: 
    # printing all the contents of the zip file 
    zf.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zf.extractall() 
    print('Done!')

del zf

File Name                                             Modified             Size
fullshpfile/city_population_data.csv           2020-08-03 09:03:56        37437
fullshpfile/HOLC_Cities.gdb/                   2020-10-07 13:29:52            0
fullshpfile/HOLC_Cities.gdb/a00000001.freelist 2020-10-07 13:29:26         4440
fullshpfile/HOLC_Cities.gdb/a00000001.gdbindexes 2020-06-11 12:28:36          110
fullshpfile/HOLC_Cities.gdb/a00000001.gdbtable 2020-10-07 13:29:26          353
fullshpfile/HOLC_Cities.gdb/a00000001.gdbtablx 2020-10-07 13:29:26         5152
fullshpfile/HOLC_Cities.gdb/a00000001.TablesByName.atx 2020-10-07 13:29:26         4118
fullshpfile/HOLC_Cities.gdb/a00000002.gdbtable 2020-06-11 12:28:36         2055
fullshpfile/HOLC_Cities.gdb/a00000002.gdbtablx 2020-06-11 12:28:36         5152
fullshpfile/HOLC_Cities.gdb/a00000003.gdbindexes 2020-06-11 12:28:36           42
fullshpfile/HOLC_Cities.gdb/a00000003.gdbtable 2020-06-11 12:28:36         1313
fullshpfile/HOLC_Cities.gdb/

In [7]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'